# MiniDataAPI Spec

The `MiniDataAPI` is a persistence API specification that designed to be small and relatively easy to implement across a wide range of datastores. While early implementations have been SQL-based, the specification can be quickly implemented in key/value stores, document databases, and more.

In [16]:
#| hide
from fasthtml.common import *

## Connect/construct the database

We connect or construct the database by passing in a string connecting to the database endpoint or a filepath representing the database's location. While this example is for a SQLite running in memory, other databases such as PostgreSQL, Redis, MongoDB, would instead use a URI pointing at the database's filepath or endpoint.

In [17]:
db = database('sqlite:///:memory:')

In [18]:
#| hide
class User: name:str; email: str; year_started:int
users = db.create(User, pk='name')

class Todo: id: int; title: str; detail: str; status: str; name: str
# If no `pk` is provided, id is assumed to be the primary key
todos = db.create(Todo) 

In [19]:
#| hide
# Remove records
[users.delete(o['name']) for o in users.rows]
[todos.delete(o['id']) for o in todos.rows]

[<Table todo (id, title, detail, status, name)>,
 <Table todo (id, title, detail, status, name)>,
 <Table todo (id, title, detail, status, name)>]

## .insert()

Add a new record to the database. Must accept Python classes, dataclasses, and dictionaries. Returns an instance of the new record.

In [20]:
class User:
    def __init__(self, name:str, email: str, year_started: int):
        self.name,self.email,self.year_started=name,email,year_started

users.insert(User(name='Braden', email='b@example.com', year_started=2018))



User(name='Braden', email='b@example.com', year_started=2018)

In [21]:
from dataclasses import dataclass

@dataclass
class UserDC: name: str; email: str; year_started: int

users.insert(UserDC(name='Alma', email='a@example.com', year_started=2019))



User(name='Alma', email='a@example.com', year_started=2019)

In [22]:
users.insert({'name': 'Charlie', 'email': 'c@example.com', 'year_started': 2018})

User(name='Charlie', email='c@example.com', year_started=2018)

In [23]:
@dataclass
class Todo: title: str; status: str; name: int

# Now the todos table
todos.insert(Todo(title='Write MiniDataAPI spec', status='open', name='Braden'))
todos.insert(Todo(title='Implement SSE in FastHTML', status='open', name='Alma'))
todo = todos.insert(Todo(title='Launch FastHTML', status='closed', name='Charlie'))
todo


Todo(id=3, title='Launch FastHTML', detail=None, status='closed', name='Charlie')

## Square bracket search []

Get a single record by entering a primary key into a table object within square brackets. 

In [24]:
users['Alma']

User(name='Alma', email='a@example.com', year_started=2019)

If no record is found, a `NotFoundError` error is raised. 

In [25]:
try: users['David']
except NotFoundError: print(f'User not found')

User not found


Here's a demonstration of a ticket search, demonstrating how this works with non-string primary keys. 

In [26]:
todos[1]

Todo(id=1, title='Write MiniDataAPI spec', detail=None, status='open', name='Braden')

## Parentheses search ()

Get zero to many records by entering values with parentheses searches. If nothing is in the parentheses, then everything is returned. 

In [27]:
for user in users():
    print(user)

User(name='Braden', email='b@example.com', year_started=2018)
User(name='Alma', email='a@example.com', year_started=2019)
User(name='Charlie', email='c@example.com', year_started=2018)


We can order the results.

In [28]:
for user in users(order_by='name'):
    print(user)

User(name='Alma', email='a@example.com', year_started=2019)
User(name='Braden', email='b@example.com', year_started=2018)
User(name='Charlie', email='c@example.com', year_started=2018)


We can filter on the results:

In [29]:
for user in users(where="year_started=2019"):
    print(user)

User(name='Alma', email='a@example.com', year_started=2019)


## .update()

Update an existing record of the database. Must accept Python dictionaries, dataclasses, and standard classes. Uses the primary key for identifying the record to be changed. Returns an instance of the updated record. 

In [30]:
users.update(User(name='Alma', year_started=1899, email='a@example.com'))


TypeError: 'User' object is not a mapping

If the primary key doesn't match a record, raise a `NoteFoundError`.

In [ ]:
try: users.update(User(name='John', year_started=2024, email='j@example.com'))
except NotFoundError: print('User not found')

User not found


## .delete()

Delete a record of the database. Uses the primary key for identifying the record to be removed. Does not return anything.

In [ ]:
try: users.delete('John')
except NotFoundError: print('User not found')

User not found


If the primary key value can't be found, raises a `NotFoundError`.

In [ ]:
users.delete('Charlie')

<Table user (name, email, year_started)>

## .xtra()

The xtra action adds a filter to queries and DDL statements. This makes it easier to limit users (or other objects) access to only things for which they have permission. 

For example, if we query all our records below, you can see todos for everyone.

In [ ]:
for o in todos():
    print(o)

Todo(id=1, title='Write MiniDataAPI spec', detail=None, status='open', name='Braden')
Todo(id=2, title='Implement SSE in FastHTML', detail=None, status='open', name='Alma')
Todo(id=3, title='Launch FastHTML', detail=None, status='closed', name='Charlie')


Now we use `.xtra` to constrain results just to Charlie.

In [ ]:
todos.xtra(name='Charlie')

Now if we loop over all the records again, only those assigned to Charlie will be displayed.

In [ ]:
for o in todos():
    print(o)

Todo(id=3, title='Launch FastHTML', detail=None, status='closed', name='Charlie')


## Implementations

- [fastlite](https://github.com/AnswerDotAI/fastlite) - The original implementation, only for Sqlite
- [fastsql](https://github.com/AnswerDotAI/fastsql) - An SQL database agnostic implementation based on the excellent SQLAlchemy library.